# Entrenamiento del Modelo de Clasificacion - Satisfaccion de Pasajeros

Este notebook guia el proceso de entrenamiento de un modelo de Machine Learning (XGBoost) para predecir la satisfaccion de los pasajeros.

## 1. Instalacion de Librerias y Carga de Datos

In [ ]:
import os
import pandas as pd

# Gestión de rutas robusta
file_name = 'airline_passenger_satisfaction.csv'
data_path = file_name
if not os.path.exists(data_path):
    # Buscar en carpeta data (estructura local)
    possible_path = os.path.join('..', 'data', file_name)
    if os.path.exists(possible_path):
        data_path = possible_path
    else:
        try:
            from google.colab import files
            print("Sube el archivo:")
            uploaded = files.upload()
            data_path = file_name
        except ImportError:
            print("Error: Archivo no encontrado.")

df = pd.read_csv(data_path)
df.head()


## 2. Preprocesamiento de Datos
Prepararemos los datos para que la IA pueda entenderlos.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

# 1. Tratar valores nulos en retrasos
df['Arrival Delay in Minutes'] = df['Arrival Delay in Minutes'].fillna(df['Departure Delay in Minutes'])

# 2. Separar caracteristicas (X) y objetivo (y)
X = df.drop('satisfaction', axis=1)
y = df['satisfaction']

# 3. Codificar la variable objetivo (satisfied=1, dissatisfied=0)
le = LabelEncoder()
y = le.fit_transform(y)

# 4. Identificar tipos de columnas
categorical_cols = X.select_dtypes(include=['object']).columns.tolist()
numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()

print(f"Variables categoricas: {categorical_cols}")
print(f"Variables numericas: {numerical_cols}")

## 3. Creacion del Pipeline y Entrenamiento

In [ ]:
from xgboost import XGBClassifier

# Pipeline para numeros: Imputar faltantes y escalar
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# Pipeline para categorias: Imputar faltantes y convertir a numeros (OneHot)
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ]
)

# Definir el modelo XGBoost
model = XGBClassifier(
    n_estimators=200,
    learning_rate=0.1,
    max_depth=6,
    random_state=42,
    eval_metric='logloss'
)

# Pipeline completo
clf = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', model)
])

# Division Train/Test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print("Entrenando modelo...")
clf.fit(X_train, y_train)
print("¡Entrenamiento completado!")

## 4. Evaluacion del Rendimiento

In [ ]:
from sklearn.metrics import classification_report, accuracy_score

train_acc = clf.score(X_train, y_train)
test_acc = clf.score(X_test, y_test)
overfitting = (train_acc - test_acc) * 100

print(f"Accuracy Entrenamiento: {train_acc:.4f}")
print(f"Accuracy Test: {test_acc:.4f}")
print(f"Overfitting: {overfitting:.2f}%")

y_pred = clf.predict(X_test)
print("\n--- Informe de Clasificacion ---")
print(classification_report(y_test, y_pred, target_names=le.classes_))

## 5. Exportar el Modelo
Guardamos el modelo para usarlo en la aplicacion Web.

In [ ]:
joblib.dump(clf, 'airline_model.joblib')
joblib.dump(le, 'label_encoder.joblib')
print("Modelos guardados como 'airline_model.joblib' y 'label_encoder.joblib'")

# Descargar los archivos a tu ordenador
files.download('airline_model.joblib')
files.download('label_encoder.joblib')